# Model assembly

## Modell Auswahl

Neuronales Netz, but WHY?????

## Modellimplementierung

In [1]:
import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [3]:
class LolProDataset(Dataset):
    def __init__(self, data_file, train):
        data = pd.read_csv(data_file)

        # Takes 90% of the data for training and 10% for testing
        split_at = int(data.shape[0]*0.9)

        if train:
            training_data = data.iloc[:split_at, :]
            self.stats = training_data.loc[:, data.columns != 'Win Rate']
            self.rates = training_data.loc[:, 'Win Rate']
        else:
            test_data = data.iloc[split_at:, :]
            self.stats = test_data.loc[:, data.columns != 'Win Rate']
            self.rates = test_data.loc[:, 'Win Rate']

    def __len__(self):
        return self.stats.shape[0]
    
    def __getitem__(self, idx):
        stats = torch.tensor(self.stats.iloc[idx], device=device, dtype=torch.float)
        rate = self.rates.iloc[idx]
        target = torch.tensor(rate, device=device, dtype=torch.float)
        sample = (stats, target)
        return sample

In [4]:
class LolProNetwork(nn.Module):
    def __init__(self, network_stack):
        super(LolProNetwork, self).__init__()
        self.network_stack = network_stack

    def forward(self, x):
        logits = self.network_stack(x)
        return logits

In [27]:
class LolProTrainer():
    def __init__(self, data_file, network_stack, learning_rate, batch_size=64, epochs=5):
        
        training_data = LolProDataset(data_file, True)
        test_data = LolProDataset(data_file, False)

        self.train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
        self.test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

        self.learning_rate = learning_rate
        self.epochs = epochs

        self.model = LolProNetwork(network_stack).to(device)

        self.loss_fn = nn.L1Loss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), self.learning_rate)

    def train_loop(self):
        size = len(self.train_dataloader.dataset)
        for batch, (X, y) in enumerate(self.train_dataloader):
            # Compute prediction and loss
            pred = self.model(X)[0]
            loss = self.loss_fn(pred, y)

            # Backpropagation
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            if batch % 100 == 0:
                loss, current = loss.item(), batch * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        

    def test_loop(self):
        size = len(self.test_dataloader.dataset)
        test_loss, err = 0, 0

        with torch.no_grad():
            for X, y in self.test_dataloader:
                pred = self.model(X)[0]
                test_loss += self.loss_fn(pred, y).item()
                err += torch.abs(pred - y).sum().data
        test_loss /= size
        err /= size
        print(f"Test Error: \n MAE: {(err):>0.1f}, Avg loss: {test_loss:>8f} \n")

    def train(self):
        for t in range(self.epochs):
            print(f"Epoch {t+1}\n-------------------------------")
            self.train_loop()
            self.test_loop()

In [32]:
network_stack = nn.Sequential(
    nn.Linear(20, 128),
    nn.Sigmoid(),
    nn.Linear(128, 64),
    nn.Sigmoid(),
    nn.Linear(64, 32),
    nn.Sigmoid(),
    nn.Linear(32, 1),
    nn.Sigmoid()
)

lr = 0.0001
batch_size = 1
epochs = 20

trainer = LolProTrainer('cleanDataTop+MidS10.csv', network_stack, lr, batch_size=batch_size, epochs=epochs)
trainer.train()

Epoch 1
-------------------------------
loss: 0.167023  [    0/  503]
loss: 0.010450  [  100/  503]
loss: 0.225640  [  200/  503]
loss: 0.256708  [  300/  503]
loss: 0.124487  [  400/  503]
loss: 0.003164  [  500/  503]
Test Error: 
 MAE: 0.1, Avg loss: 0.124991 

Epoch 2
-------------------------------
loss: 0.112885  [    0/  503]
loss: 0.104678  [  100/  503]
loss: 0.364927  [  200/  503]
loss: 0.220969  [  300/  503]
loss: 0.051484  [  400/  503]
loss: 0.118358  [  500/  503]
Test Error: 
 MAE: 0.1, Avg loss: 0.116816 

Epoch 3
-------------------------------
loss: 0.215358  [    0/  503]
loss: 0.317169  [  100/  503]
loss: 0.102158  [  200/  503]
loss: 0.247655  [  300/  503]
loss: 0.223583  [  400/  503]
loss: 0.026645  [  500/  503]
Test Error: 
 MAE: 0.1, Avg loss: 0.109980 

Epoch 4
-------------------------------
loss: 0.053659  [    0/  503]
loss: 0.264892  [  100/  503]
loss: 0.155048  [  200/  503]
loss: 0.183323  [  300/  503]
loss: 0.004503  [  400/  503]
loss: 0.029665 

In [31]:
trainer.model.forward(torch.tensor([[100, 5, 4, 1.5, 3.5, 9, 450, 0.5, 0.2, 1.3, 0.6, 0.2, 0.3, 500, 400, 0.3, 0.05, 40]], device=device, dtype=float))

RuntimeError: mat1 dim 1 must match mat2 dim 0